### importing modules

In [22]:
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['font.size'] = 15

In [23]:
import numpy as np
import gym
from collections import deque
import random
import pickle
import time
import copy

In [24]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

## class for DQN

In [25]:
class DQN:
    
    def __init__(self, o_space, a_space, lr, units=32, mem_len=10000):
        self.a_space = a_space
        self.state_size = o_space

        self.lr = lr
        self.units = units
        
        # hyperparameters
        self.discount = 0.99
        self.epsilon = 1.0
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.01
        self.batch_size = 50
        self.train_start = 1000
        self.memory = deque(maxlen=10000)

        # create main model and target model
        self.model = self.init_model()
        self.target_model = self.init_model()

        # initialize target model
        self.update_target_model()

    def init_model(self):
        model = Sequential()
        model.add(Dense(self.units, input_dim=self.state_size, activation='relu',
                        kernel_initializer='he_uniform'))
        model.add(Dense(self.units, activation='relu',
                        kernel_initializer='he_uniform'))
        model.add(Dense(self.a_space, activation='linear',
                        kernel_initializer='he_uniform'))
        model.compile(loss='mse', optimizer=Adam(lr=self.lr))
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def get_action(self, s):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.a_space)
        else:
            q_value = self.model.predict(s)
            return np.argmax(q_value[0])

    def add_experience(self, s, a, r, s_next, terminal):
        self.memory.append([s, a, r, s_next, terminal])
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
      
    def train_model(self):
        if len(self.memory) < self.train_start:
            return
        batch_size = min(self.batch_size, len(self.memory))
        mini_batch = random.sample(self.memory, batch_size)

        update_input = np.zeros((batch_size, self.state_size))
        update_target = np.zeros((batch_size, self.state_size))
        a, r, terminal = [], [], []

        for i in range(self.batch_size):
            update_input[i] = mini_batch[i][0]
            a.append(mini_batch[i][1])
            r.append(mini_batch[i][2])
            update_target[i] = mini_batch[i][3]
            terminal.append(mini_batch[i][4])

        target = self.model.predict(update_input)
        target_val = self.target_model.predict(update_target)

        for i in range(self.batch_size):
            if terminal[i]:
                target[i][a[i]] = r[i]
            else:
                target[i][a[i]] = r[i] + self.discount * (np.amax(target_val[i]))

        self.model.fit(update_input, target, batch_size=self.batch_size, epochs=1, verbose=0) 

In [26]:
terminal_r_dict = {
    'CartPole-v1': -1.0,
    'LunarLander-v2': 100.0,
    'MountainCar-v0': 1.0
}

memory_env = {
    'CartPole-v1': 10000,
    'LunarLander-v2': 10000,
    'MountainCar-v0': 10000
}

In [27]:
class DQN_HER:
    
    def __init__(self, o_space, a_space, lr, units=32, mem_len=10000):
        self.a_space = a_space
        self.state_size = o_space

        self.lr = lr
        self.units = units
        # hyperparameters
        self.discount = 0.99
        self.epsilon = 1.0
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.01
        self.batch_size = 50
        self.train_start = 1000
        self.memory = deque(maxlen=mem_len)
        self.her_memory = deque(maxlen=1000)

        # create main model and target model
        self.model = self.init_model()
        self.target_model = self.init_model()

        # initialize target model
        self.update_target_model()

    def init_model(self):
        model = Sequential()
        model.add(Dense(self.units, input_dim=self.state_size, activation='relu',
                        kernel_initializer='he_uniform'))
        model.add(Dense(self.units, activation='relu',
                        kernel_initializer='he_uniform'))
        model.add(Dense(self.a_space, activation='linear',
                        kernel_initializer='he_uniform'))
        model.compile(loss='mse', optimizer=Adam(lr=self.lr))
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def get_action(self, s):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.a_space)
        else:
            q_value = self.model.predict(s)
            return np.argmax(q_value[0])

    def add_experience(self, s, a, r, s_next, terminal):
        self.memory.append([s, a, r, s_next, terminal])
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def add_her_experience(self, s, a, r, s_next, terminal):
        self.her_memory.append([s, a, r, s_next, terminal])
        
    def modify_her_list(self, terminal_reward):
        new_her_experience = copy.deepcopy(self.her_memory)
        new_her_len = len(new_her_experience)
        her_goal = self.her_memory[-1][3]
        
        for i in range(new_her_len):
            new_her_experience[-1-i][0] = her_goal
            new_her_experience[-1-i][2] = self.memory[-1-i][2]
            new_her_experience[-1-i][3] = her_goal
            new_her_experience[-1-i][4] = False
            
            if (np.sum(np.abs((new_her_experience[-1-i][3] - her_goal))) == 0):
                new_her_experience[-1-i][2] = terminal_reward
                new_her_experience[-1-i][4] = True
        
        for hx in new_her_experience:
            self.memory.append(hx)
            
        self.reset_her()
            
    def reset_her(self):
        self.her_memory.clear()
      
    def train_model(self):
        if len(self.memory) < self.train_start:
            return
        batch_size = min(self.batch_size, len(self.memory))
        mini_batch = random.sample(self.memory, batch_size)

        update_input = np.zeros((batch_size, self.state_size))
        update_target = np.zeros((batch_size, self.state_size))
        a, r, terminal = [], [], []

        for i in range(self.batch_size):
            update_input[i] = mini_batch[i][0]
            a.append(mini_batch[i][1])
            r.append(mini_batch[i][2])
            update_target[i] = mini_batch[i][3]
            terminal.append(mini_batch[i][4])

        target = self.model.predict(update_input)
        target_val = self.target_model.predict(update_target)

        for i in range(self.batch_size):
            if terminal[i]:
                target[i][a[i]] = r[i]
            else:
                target[i][a[i]] = r[i] + self.discount * (np.amax(target_val[i]))

        self.model.fit(update_input, target, batch_size=self.batch_size, epochs=1, verbose=0) 

## main

#### settings

In [55]:
lr = 0.001
num_seeds = 10
seeds = np.arange(num_seeds)
num_episodes = 500

### classic replay

In [56]:
%%time

result_dict = {}

env_name = 'CartPole-v1'

for seed in seeds:
    if seed not in result_dict.keys():
        result_dict[seed] = []
    
    env = gym.make(env_name)
    
    np.random.seed(seed)
    env.action_space.np_random.seed(seed)
    
    o_space = env.observation_space.shape[0]
    a_space = env.action_space.n
    agent = DQN(o_space, a_space, lr, units=32)
    
    scores, episodes = [], []
    
    for num_episode in range(num_episodes):
        terminal = False
        score = 0
        s = env.reset()
        s = np.reshape(s, [1, o_space])

        while not terminal:

            a = agent.get_action(s)
            s_next, r, terminal, _ = env.step(a)
            s_next = np.reshape(s_next, [1, o_space])

            # if an action make the episode end, then gives penalty of -100

            # save the sample <s, a, r, s'> to the replay memory
            agent.add_experience(s, a, r, s_next, terminal)
            # every time step do the training
            agent.train_model()
            score += r
            s = s_next

            if terminal:
                agent.update_target_model()

                scores.append(score)
                episodes.append(num_episode)
                print("episode:", num_episode, "  score:", score, "  memory length:",
                      len(agent.memory), "  epsilon:", agent.epsilon)
    
    result_dict[seed] = scores
        
print(result_dict)

episode: 0   score: 23.0   memory length: 23   epsilon: 0.9772512378214517
episode: 1   score: 30.0   memory length: 53   epsilon: 0.9483548639781193
episode: 2   score: 12.0   memory length: 65   epsilon: 0.9370369888620198
episode: 3   score: 23.0   memory length: 88   epsilon: 0.9157205572498949
episode: 4   score: 14.0   memory length: 102   epsilon: 0.9029834676116293
episode: 5   score: 17.0   memory length: 119   epsilon: 0.887754942528593
episode: 6   score: 12.0   memory length: 131   epsilon: 0.8771602801771059
episode: 7   score: 13.0   memory length: 144   epsilon: 0.8658253647948594
episode: 8   score: 15.0   memory length: 159   epsilon: 0.8529285032149548
episode: 9   score: 19.0   memory length: 178   epsilon: 0.8368678892362568
episode: 10   score: 13.0   memory length: 191   epsilon: 0.8260536436246144
episode: 11   score: 19.0   memory length: 210   epsilon: 0.8104990823150267
episode: 12   score: 19.0   memory length: 229   epsilon: 0.7952374128525983
episode: 13   

episode: 108   score: 14.0   memory length: 1482   epsilon: 0.22701447560002463
episode: 109   score: 9.0   memory length: 1491   epsilon: 0.2249794988001053
episode: 110   score: 10.0   memory length: 1501   epsilon: 0.22273980093919937
episode: 111   score: 11.0   memory length: 1512   epsilon: 0.22030187713925398
episode: 112   score: 16.0   memory length: 1528   epsilon: 0.21680336036122028
episode: 113   score: 14.0   memory length: 1542   epsilon: 0.21378776372211966
episode: 114   score: 31.0   memory length: 1573   epsilon: 0.20725880007153075
episode: 115   score: 46.0   memory length: 1619   epsilon: 0.1979362954770861
episode: 116   score: 92.0   memory length: 1711   epsilon: 0.1805304042562526
episode: 117   score: 90.0   memory length: 1801   epsilon: 0.1649849368967147
episode: 118   score: 127.0   memory length: 1928   epsilon: 0.14529855648899012
episode: 119   score: 37.0   memory length: 1965   epsilon: 0.1400181593009795
episode: 120   score: 70.0   memory length: 2

episode: 209   score: 178.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 210   score: 199.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 211   score: 169.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 212   score: 193.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 213   score: 189.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 214   score: 162.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 215   score: 172.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 216   score: 163.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 217   score: 190.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 218   score: 159.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 219   score: 168.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 220   score: 177.0   memory length: 10000   epsilon: 0.009998671593271896
epis

episode: 308   score: 328.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 309   score: 292.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 310   score: 231.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 311   score: 205.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 312   score: 322.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 313   score: 500.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 314   score: 500.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 315   score: 500.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 316   score: 500.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 317   score: 500.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 318   score: 500.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 319   score: 500.0   memory length: 10000   epsilon: 0.009998671593271896
epis

episode: 407   score: 87.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 408   score: 69.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 409   score: 71.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 410   score: 73.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 411   score: 73.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 412   score: 47.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 413   score: 48.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 414   score: 33.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 415   score: 44.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 416   score: 103.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 417   score: 96.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 418   score: 125.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 419  

...

episode: 94   score: 16.0   memory length: 1230   epsilon: 0.29211275315428575
episode: 95   score: 13.0   memory length: 1243   epsilon: 0.28833798882226386
episode: 96   score: 9.0   memory length: 1252   epsilon: 0.2857533029063643
episode: 97   score: 12.0   memory length: 1264   epsilon: 0.2823430602649749
episode: 98   score: 10.0   memory length: 1274   epsilon: 0.2795323012780908
episode: 99   score: 12.0   memory length: 1286   epsilon: 0.2761963014356792
episode: 100   score: 15.0   memory length: 1301   epsilon: 0.2720822322326576
episode: 101   score: 10.0   memory length: 1311   epsilon: 0.26937362101798235
episode: 102   score: 10.0   memory length: 1321   epsilon: 0.2666919743524145
episode: 103   score: 28.0   memory length: 1349   epsilon: 0.259324540388352
episode: 104   score: 12.0   memory length: 1361   epsilon: 0.256229704400119
episode: 105   score: 11.0   memory length: 1372   epsilon: 0.253425228091996
episode: 106   score: 11.0   memory length: 1383   epsilon:

episode: 195   score: 206.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 196   score: 215.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 197   score: 173.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 198   score: 225.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 199   score: 249.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 200   score: 240.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 201   score: 207.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 202   score: 190.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 203   score: 221.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 204   score: 282.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 205   score: 218.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 206   score: 313.0   memory length: 10000   epsilon: 0.009998671593271896
epis

episode: 294   score: 489.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 295   score: 412.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 296   score: 329.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 297   score: 236.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 298   score: 227.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 299   score: 336.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 300   score: 176.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 301   score: 184.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 302   score: 444.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 303   score: 279.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 304   score: 203.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 305   score: 478.0   memory length: 10000   epsilon: 0.009998671593271896
epis

episode: 393   score: 223.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 394   score: 193.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 395   score: 192.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 396   score: 255.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 397   score: 236.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 398   score: 204.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 399   score: 200.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 400   score: 246.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 401   score: 500.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 402   score: 354.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 403   score: 316.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 404   score: 235.0   memory length: 10000   epsilon: 0.009998671593271896
epis

episode: 492   score: 500.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 493   score: 305.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 494   score: 433.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 495   score: 137.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 496   score: 500.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 497   score: 500.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 498   score: 418.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 499   score: 359.0   memory length: 10000   epsilon: 0.009998671593271896
{0: [23.0, 30.0, 12.0, 23.0, 14.0, 17.0, 12.0, 13.0, 15.0, 19.0, 13.0, 19.0, 19.0, 36.0, 11.0, 11.0, 13.0, 12.0, 16.0, 18.0, 13.0, 13.0, 48.0, 24.0, 26.0, 15.0, 15.0, 12.0, 10.0, 15.0, 10.0, 11.0, 16.0, 10.0, 16.0, 21.0, 15.0, 10.0, 14.0, 23.0, 13.0, 10.0, 19.0, 12.0, 10.0, 18.0, 10.0, 9.0, 19.0, 13.0, 16.0, 10.0, 11.0, 17.0, 15.0, 16

In [57]:
%%time

result_dict = {}

env_name = 'MountainCar-v0'

for seed in seeds:
    if seed not in result_dict.keys():
        result_dict[seed] = []
    
    env = gym.make(env_name)
    
    np.random.seed(seed)
    env.action_space.np_random.seed(seed)
    
    o_space = env.observation_space.shape[0]
    a_space = env.action_space.n
    agent = DQN(o_space, a_space, lr, units=64)
    
    scores, episodes = [], []
    
    for num_episode in range(num_episodes):
        terminal = False
        score = 0
        s = env.reset()
        s = np.reshape(s, [1, o_space])

        while not terminal:

            a = agent.get_action(s)
            s_next, r, terminal, _ = env.step(a)
            s_next = np.reshape(s_next, [1, o_space])

            # if an action make the episode end, then gives penalty of -100

            # save the sample <s, a, r, s'> to the replay memory
            agent.add_experience(s, a, r, s_next, terminal)
            # every time step do the training
            agent.train_model()
            score += r
            s = s_next

            if terminal:
                agent.update_target_model()

                scores.append(score)
                episodes.append(num_episode)
                print("episode:", num_episode, "  score:", score, "  memory length:",
                      len(agent.memory), "  epsilon:", agent.epsilon)
    
    result_dict[seed] = scores
        
print(result_dict)

episode: 0   score: -200.0   memory length: 200   epsilon: 0.818648829478636
episode: 1   score: -200.0   memory length: 400   epsilon: 0.6701859060067403
episode: 2   score: -200.0   memory length: 600   epsilon: 0.5486469074854965
episode: 3   score: -200.0   memory length: 800   epsilon: 0.4491491486100748
episode: 4   score: -200.0   memory length: 1000   epsilon: 0.3676954247709635
episode: 5   score: -200.0   memory length: 1200   epsilon: 0.3010134290933992
episode: 6   score: -200.0   memory length: 1400   epsilon: 0.24642429138466176
episode: 7   score: -200.0   memory length: 1600   epsilon: 0.20173495769715546
episode: 8   score: -200.0   memory length: 1800   epsilon: 0.1651500869836984
episode: 9   score: -200.0   memory length: 2000   epsilon: 0.1351999253974994
episode: 10   score: -200.0   memory length: 2200   epsilon: 0.11068126067226178
episode: 11   score: -200.0   memory length: 2400   epsilon: 0.09060908449456685
episode: 12   score: -200.0   memory length: 2600  

episode: 100   score: -181.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 101   score: -172.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 102   score: -164.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 103   score: -155.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 104   score: -155.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 105   score: -149.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 106   score: -172.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 107   score: -170.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 108   score: -167.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 109   score: -160.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 110   score: -172.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 111   score: -173.0   memory length: 10000   epsilon: 0.00999867159

episode: 198   score: -148.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 199   score: -137.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 200   score: -140.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 201   score: -138.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 202   score: -145.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 203   score: -156.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 204   score: -143.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 205   score: -143.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 206   score: -135.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 207   score: -140.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 208   score: -138.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 209   score: -140.0   memory length: 10000   epsilon: 0.00999867159

episode: 296   score: -149.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 297   score: -146.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 298   score: -145.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 299   score: -90.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 300   score: -155.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 301   score: -84.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 302   score: -144.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 303   score: -94.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 304   score: -146.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 305   score: -87.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 306   score: -89.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 307   score: -86.0   memory length: 10000   epsilon: 0.00999867159327189

episode: 394   score: -107.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 395   score: -96.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 396   score: -175.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 397   score: -105.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 398   score: -90.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 399   score: -87.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 400   score: -160.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 401   score: -86.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 402   score: -115.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 403   score: -104.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 404   score: -141.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 405   score: -150.0   memory length: 10000   epsilon: 0.009998671593271

...

episode: 33   score: -200.0   memory length: 6800   epsilon: 0.009998671593271896
episode: 34   score: -200.0   memory length: 7000   epsilon: 0.009998671593271896
episode: 35   score: -200.0   memory length: 7200   epsilon: 0.009998671593271896
episode: 36   score: -200.0   memory length: 7400   epsilon: 0.009998671593271896
episode: 37   score: -200.0   memory length: 7600   epsilon: 0.009998671593271896
episode: 38   score: -145.0   memory length: 7745   epsilon: 0.009998671593271896
episode: 39   score: -200.0   memory length: 7945   epsilon: 0.009998671593271896
episode: 40   score: -183.0   memory length: 8128   epsilon: 0.009998671593271896
episode: 41   score: -200.0   memory length: 8328   epsilon: 0.009998671593271896
episode: 42   score: -200.0   memory length: 8528   epsilon: 0.009998671593271896
episode: 43   score: -200.0   memory length: 8728   epsilon: 0.009998671593271896
episode: 44   score: -200.0   memory length: 8928   epsilon: 0.009998671593271896
episode: 45   sc

episode: 132   score: -151.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 133   score: -154.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 134   score: -159.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 135   score: -161.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 136   score: -102.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 137   score: -147.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 138   score: -175.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 139   score: -169.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 140   score: -91.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 141   score: -153.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 142   score: -141.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 143   score: -178.0   memory length: 10000   epsilon: 0.009998671593

episode: 230   score: -144.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 231   score: -141.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 232   score: -146.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 233   score: -140.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 234   score: -129.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 235   score: -142.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 236   score: -87.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 237   score: -90.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 238   score: -103.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 239   score: -148.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 240   score: -157.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 241   score: -138.0   memory length: 10000   epsilon: 0.0099986715932

episode: 328   score: -153.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 329   score: -150.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 330   score: -150.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 331   score: -145.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 332   score: -101.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 333   score: -153.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 334   score: -154.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 335   score: -146.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 336   score: -96.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 337   score: -86.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 338   score: -174.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 339   score: -151.0   memory length: 10000   epsilon: 0.0099986715932

episode: 426   score: -144.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 427   score: -103.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 428   score: -103.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 429   score: -151.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 430   score: -139.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 431   score: -155.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 432   score: -145.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 433   score: -141.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 434   score: -92.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 435   score: -97.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 436   score: -140.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 437   score: -140.0   memory length: 10000   epsilon: 0.0099986715932

In [58]:
%%time

result_dict = {}

env_name = 'LunarLander-v2'

for seed in seeds:
    if seed not in result_dict.keys():
        result_dict[seed] = []
    
    env = gym.make(env_name)
    
    np.random.seed(seed)
    env.action_space.np_random.seed(seed)
    
    o_space = env.observation_space.shape[0]
    a_space = env.action_space.n
    agent = DQN(o_space, a_space, lr, units=128)
    
    scores, episodes = [], []
    
    for num_episode in range(num_episodes):
        terminal = False
        score = 0
        s = env.reset()
        s = np.reshape(s, [1, o_space])

        while not terminal:

            a = agent.get_action(s)
            s_next, r, terminal, _ = env.step(a)
            s_next = np.reshape(s_next, [1, o_space])

            # if an action make the episode end, then gives penalty of -100

            # save the sample <s, a, r, s'> to the replay memory
            agent.add_experience(s, a, r, s_next, terminal)
            # every time step do the training
            agent.train_model()
            score += r
            s = s_next

            if terminal:
                agent.update_target_model()

                scores.append(score)
                episodes.append(num_episode)
                print("episode:", num_episode, "  score:", score, "  memory length:",
                      len(agent.memory), "  epsilon:", agent.epsilon)
    
    result_dict[seed] = scores
        
print(result_dict)

episode: 0   score: -105.73637539219824   memory length: 64   epsilon: 0.9379749638258457
episode: 1   score: -279.2038802864672   memory length: 163   epsilon: 0.8495219033622532
episode: 2   score: -422.2187524496111   memory length: 252   epsilon: 0.7771467460721305
episode: 3   score: -252.78896238724397   memory length: 320   epsilon: 0.7260327850203407
episode: 4   score: -282.40551244256494   memory length: 406   epsilon: 0.6661748299656206
episode: 5   score: -289.04306299049824   memory length: 472   epsilon: 0.623606226269926
episode: 6   score: -372.3548237724995   memory length: 543   epsilon: 0.5808448013500086
episode: 7   score: -402.80578696153617   memory length: 605   epsilon: 0.5459091539334175
episode: 8   score: -532.5273298937791   memory length: 688   epsilon: 0.5024072518560504
episode: 9   score: -310.3956934043989   memory length: 745   epsilon: 0.47455737640838375
episode: 10   score: -497.0439284418464   memory length: 811   epsilon: 0.44423313721693997
epis

episode: 87   score: -33.55507514431528   memory length: 10000   epsilon: 0.009998671593271896
episode: 88   score: 168.49125249185602   memory length: 10000   epsilon: 0.009998671593271896
episode: 89   score: -82.88247936212736   memory length: 10000   epsilon: 0.009998671593271896
episode: 90   score: -118.64380783571596   memory length: 10000   epsilon: 0.009998671593271896
episode: 91   score: -44.30993599685888   memory length: 10000   epsilon: 0.009998671593271896
episode: 92   score: 8.339285803442436   memory length: 10000   epsilon: 0.009998671593271896
episode: 93   score: 172.02040694136707   memory length: 10000   epsilon: 0.009998671593271896
episode: 94   score: 237.4255656264892   memory length: 10000   epsilon: 0.009998671593271896
episode: 95   score: -39.08641837257983   memory length: 10000   epsilon: 0.009998671593271896
episode: 96   score: 198.88414142328116   memory length: 10000   epsilon: 0.009998671593271896
episode: 97   score: 217.06646765171752   memory le

episode: 173   score: -210.55494929378654   memory length: 10000   epsilon: 0.009998671593271896
episode: 174   score: -61.03014959376882   memory length: 10000   epsilon: 0.009998671593271896
episode: 175   score: 228.72747936968986   memory length: 10000   epsilon: 0.009998671593271896
episode: 176   score: 187.12374041134342   memory length: 10000   epsilon: 0.009998671593271896
episode: 177   score: -38.046096617070305   memory length: 10000   epsilon: 0.009998671593271896
episode: 178   score: -96.19037176114053   memory length: 10000   epsilon: 0.009998671593271896
episode: 179   score: -80.01851906672987   memory length: 10000   epsilon: 0.009998671593271896
episode: 180   score: -155.26823585756992   memory length: 10000   epsilon: 0.009998671593271896
episode: 181   score: 195.40125140522477   memory length: 10000   epsilon: 0.009998671593271896
episode: 182   score: 2.6366852304959707   memory length: 10000   epsilon: 0.009998671593271896
episode: 183   score: -120.5349385647

episode: 259   score: 266.9029739504291   memory length: 10000   epsilon: 0.009998671593271896
episode: 260   score: 240.47463473269843   memory length: 10000   epsilon: 0.009998671593271896
episode: 261   score: 226.08553422506333   memory length: 10000   epsilon: 0.009998671593271896
episode: 262   score: 301.11360034639625   memory length: 10000   epsilon: 0.009998671593271896
episode: 263   score: 290.83642079059433   memory length: 10000   epsilon: 0.009998671593271896
episode: 264   score: 282.5946845333133   memory length: 10000   epsilon: 0.009998671593271896
episode: 265   score: 136.80993178190698   memory length: 10000   epsilon: 0.009998671593271896
episode: 266   score: 273.9388837553225   memory length: 10000   epsilon: 0.009998671593271896
episode: 267   score: 254.3189210191104   memory length: 10000   epsilon: 0.009998671593271896
episode: 268   score: 266.1818190049306   memory length: 10000   epsilon: 0.009998671593271896
episode: 269   score: 287.19236261244794   me

episode: 345   score: 229.60720795143044   memory length: 10000   epsilon: 0.009998671593271896
episode: 346   score: 9.502975813384921   memory length: 10000   epsilon: 0.009998671593271896
episode: 347   score: 254.5566676408245   memory length: 10000   epsilon: 0.009998671593271896
episode: 348   score: 238.61334886212134   memory length: 10000   epsilon: 0.009998671593271896
episode: 349   score: 270.1516287856824   memory length: 10000   epsilon: 0.009998671593271896
episode: 350   score: 262.9162611057591   memory length: 10000   epsilon: 0.009998671593271896
episode: 351   score: 291.0980776428392   memory length: 10000   epsilon: 0.009998671593271896
episode: 352   score: 273.72299985403197   memory length: 10000   epsilon: 0.009998671593271896
episode: 353   score: 253.3647227878405   memory length: 10000   epsilon: 0.009998671593271896
episode: 354   score: 251.53384403920631   memory length: 10000   epsilon: 0.009998671593271896
episode: 355   score: 289.02771889691536   mem

...

episode: 70   score: 181.41772901335466   memory length: 10000   epsilon: 0.009998671593271896
episode: 71   score: 202.37377015204692   memory length: 10000   epsilon: 0.009998671593271896
episode: 72   score: -10.942063486305857   memory length: 10000   epsilon: 0.009998671593271896
episode: 73   score: -77.82633253929741   memory length: 10000   epsilon: 0.009998671593271896
episode: 74   score: 175.67910290132522   memory length: 10000   epsilon: 0.009998671593271896
episode: 75   score: 167.69617932070497   memory length: 10000   epsilon: 0.009998671593271896
episode: 76   score: -14.300702792169888   memory length: 10000   epsilon: 0.009998671593271896
episode: 77   score: -15.49948559087765   memory length: 10000   epsilon: 0.009998671593271896
episode: 78   score: -34.46877422491573   memory length: 10000   epsilon: 0.009998671593271896
episode: 79   score: -10.7189737666951   memory length: 10000   epsilon: 0.009998671593271896
episode: 80   score: -26.537098578551255   memory

episode: 156   score: 187.50901108251165   memory length: 10000   epsilon: 0.009998671593271896
episode: 157   score: -52.830453956083126   memory length: 10000   epsilon: 0.009998671593271896
episode: 158   score: -45.43999864507567   memory length: 10000   epsilon: 0.009998671593271896
episode: 159   score: -90.2206338042214   memory length: 10000   epsilon: 0.009998671593271896
episode: 160   score: -463.9394544853245   memory length: 10000   epsilon: 0.009998671593271896
episode: 161   score: -141.47606021440572   memory length: 10000   epsilon: 0.009998671593271896
episode: 162   score: -243.19015875568996   memory length: 10000   epsilon: 0.009998671593271896
episode: 163   score: -216.96556409679295   memory length: 10000   epsilon: 0.009998671593271896
episode: 164   score: -210.43295633594488   memory length: 10000   epsilon: 0.009998671593271896
episode: 165   score: 199.87718540505617   memory length: 10000   epsilon: 0.009998671593271896
episode: 166   score: -142.316499546

episode: 242   score: 240.2754289100553   memory length: 10000   epsilon: 0.009998671593271896
episode: 243   score: -514.8125643027089   memory length: 10000   epsilon: 0.009998671593271896
episode: 244   score: 269.6576451631339   memory length: 10000   epsilon: 0.009998671593271896
episode: 245   score: -434.25131019766087   memory length: 10000   epsilon: 0.009998671593271896
episode: 246   score: 267.54130601230514   memory length: 10000   epsilon: 0.009998671593271896
episode: 247   score: 228.08211312650053   memory length: 10000   epsilon: 0.009998671593271896
episode: 248   score: 248.64529148984013   memory length: 10000   epsilon: 0.009998671593271896
episode: 249   score: -209.79972663603962   memory length: 10000   epsilon: 0.009998671593271896
episode: 250   score: 270.9833261611916   memory length: 10000   epsilon: 0.009998671593271896
episode: 251   score: 258.65702531256744   memory length: 10000   epsilon: 0.009998671593271896
episode: 252   score: 267.353728623813   

episode: 328   score: -67.41238703861677   memory length: 10000   epsilon: 0.009998671593271896
episode: 329   score: 280.43386377681674   memory length: 10000   epsilon: 0.009998671593271896
episode: 330   score: 269.0647398938415   memory length: 10000   epsilon: 0.009998671593271896
episode: 331   score: 242.35607283735368   memory length: 10000   epsilon: 0.009998671593271896
episode: 332   score: -9.822540017804556   memory length: 10000   epsilon: 0.009998671593271896
episode: 333   score: 238.53095059497778   memory length: 10000   epsilon: 0.009998671593271896
episode: 334   score: 257.6988722675485   memory length: 10000   epsilon: 0.009998671593271896
episode: 335   score: 269.46566541614015   memory length: 10000   epsilon: 0.009998671593271896
episode: 336   score: 251.7518944698003   memory length: 10000   epsilon: 0.009998671593271896
episode: 337   score: 235.1435375586184   memory length: 10000   epsilon: 0.009998671593271896
episode: 338   score: 280.1548336391397   me

episode: 414   score: 228.7459007224764   memory length: 10000   epsilon: 0.009998671593271896
episode: 415   score: 227.35112848125348   memory length: 10000   epsilon: 0.009998671593271896
episode: 416   score: 157.6369406501322   memory length: 10000   epsilon: 0.009998671593271896
episode: 417   score: 232.47882370981668   memory length: 10000   epsilon: 0.009998671593271896
episode: 418   score: 269.35624301046903   memory length: 10000   epsilon: 0.009998671593271896
episode: 419   score: 266.11606533644317   memory length: 10000   epsilon: 0.009998671593271896
episode: 420   score: 260.1706635407478   memory length: 10000   epsilon: 0.009998671593271896
episode: 421   score: 277.0904228321508   memory length: 10000   epsilon: 0.009998671593271896
episode: 422   score: 13.406986179884811   memory length: 10000   epsilon: 0.009998671593271896
episode: 423   score: 255.10871807399016   memory length: 10000   epsilon: 0.009998671593271896
episode: 424   score: 267.06012631380446   m

In [83]:
%%time

seeds = np.arange(5)

result_dict = {}

env_name = 'Acrobot-v1'

for seed in seeds:
    if seed not in result_dict.keys():
        result_dict[seed] = []
    
    env = gym.make(env_name)
    
    np.random.seed(seed)
    env.action_space.np_random.seed(seed)
    
    o_space = env.observation_space.shape[0]
    a_space = env.action_space.n
    agent = DQN(o_space, a_space, lr, units=128)
    
    scores, episodes = [], []
    
    for num_episode in range(num_episodes):
        terminal = False
        score = 0
        s = env.reset()
        s = np.reshape(s, [1, o_space])

        while not terminal:

            a = agent.get_action(s)
            s_next, r, terminal, _ = env.step(a)
            s_next = np.reshape(s_next, [1, o_space])

            # if an action make the episode end, then gives penalty of -100

            # save the sample <s, a, r, s'> to the replay memory
            agent.add_experience(s, a, r, s_next, terminal)
            # every time step do the training
            agent.train_model()
            score += r
            s = s_next

            if terminal:
                agent.update_target_model()

                scores.append(score)
                episodes.append(num_episode)
                print("episode:", num_episode, "  score:", score, "  memory length:",
                      len(agent.memory), "  epsilon:", agent.epsilon)
    
    result_dict[seed] = scores
        
print(result_dict)

episode: 0   score: -358.0   memory length: 359   epsilon: 0.698248920785887
episode: 1   score: -360.0   memory length: 720   epsilon: 0.4865769398194536
episode: 2   score: -312.0   memory length: 1033   epsilon: 0.35575362775079855
episode: 3   score: -500.0   memory length: 1533   epsilon: 0.21572150942606796
episode: 4   score: -315.0   memory length: 1849   epsilon: 0.1572489411773005
episode: 5   score: -334.0   memory length: 2184   epsilon: 0.1124673042411007
episode: 6   score: -207.0   memory length: 2392   epsilon: 0.09133723000062824
episode: 7   score: -232.0   memory length: 2625   epsilon: 0.07234467887111937
episode: 8   score: -500.0   memory length: 3125   epsilon: 0.043868290040190716
episode: 9   score: -284.0   memory length: 3410   epsilon: 0.03298487560678867
episode: 10   score: -259.0   memory length: 3670   epsilon: 0.02542973235819944
episode: 11   score: -487.0   memory length: 4158   epsilon: 0.015606303325022058
episode: 12   score: -464.0   memory length

episode: 100   score: -92.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 101   score: -101.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 102   score: -99.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 103   score: -76.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 104   score: -75.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 105   score: -71.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 106   score: -90.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 107   score: -63.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 108   score: -86.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 109   score: -93.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 110   score: -80.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 111   score: -63.0   memory length: 10000   epsilon: 0.009998671593271896
epi

episode: 199   score: -74.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 200   score: -86.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 201   score: -114.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 202   score: -86.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 203   score: -82.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 204   score: -110.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 205   score: -108.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 206   score: -93.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 207   score: -62.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 208   score: -109.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 209   score: -62.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 210   score: -132.0   memory length: 10000   epsilon: 0.009998671593271896

episode: 298   score: -72.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 299   score: -63.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 300   score: -73.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 301   score: -95.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 302   score: -62.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 303   score: -79.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 304   score: -104.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 305   score: -71.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 306   score: -89.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 307   score: -101.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 308   score: -161.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 309   score: -86.0   memory length: 10000   epsilon: 0.009998671593271896
e

episode: 397   score: -89.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 398   score: -63.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 399   score: -70.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 400   score: -79.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 401   score: -103.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 402   score: -70.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 403   score: -108.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 404   score: -104.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 405   score: -90.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 406   score: -93.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 407   score: -75.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 408   score: -62.0   memory length: 10000   epsilon: 0.009998671593271896
e

...

episode: 84   score: -119.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 85   score: -77.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 86   score: -131.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 87   score: -84.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 88   score: -77.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 89   score: -105.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 90   score: -103.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 91   score: -62.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 92   score: -126.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 93   score: -91.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 94   score: -71.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 95   score: -93.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 96

episode: 183   score: -98.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 184   score: -69.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 185   score: -61.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 186   score: -83.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 187   score: -105.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 188   score: -87.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 189   score: -65.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 190   score: -85.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 191   score: -71.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 192   score: -79.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 193   score: -75.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 194   score: -95.0   memory length: 10000   epsilon: 0.009998671593271896
epi

episode: 282   score: -64.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 283   score: -122.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 284   score: -80.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 285   score: -86.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 286   score: -75.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 287   score: -96.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 288   score: -92.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 289   score: -71.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 290   score: -115.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 291   score: -95.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 292   score: -126.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 293   score: -100.0   memory length: 10000   epsilon: 0.009998671593271896


episode: 381   score: -93.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 382   score: -63.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 383   score: -77.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 384   score: -64.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 385   score: -64.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 386   score: -93.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 387   score: -63.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 388   score: -72.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 389   score: -62.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 390   score: -88.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 391   score: -165.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 392   score: -70.0   memory length: 10000   epsilon: 0.009998671593271896
epi

episode: 480   score: -101.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 481   score: -109.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 482   score: -63.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 483   score: -84.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 484   score: -62.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 485   score: -93.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 486   score: -62.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 487   score: -95.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 488   score: -71.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 489   score: -64.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 490   score: -85.0   memory length: 10000   epsilon: 0.009998671593271896
episode: 491   score: -87.0   memory length: 10000   epsilon: 0.009998671593271896
ep